In [1]:
import numpy as np
import pandas as pd

In [2]:
#load data set
file_path = "C:/Users/raoki/Documents/GitHub/project_spring2019/Data/tcga_rna.txt"
data = pd.read_csv(file_path,sep=';')

In [3]:
data = data[0:100]

In [4]:
# generate dataset with first nums_gene features
# create both patient/gene id table to map patient/gene information
# create long version of gene dataset

def sample_data(df, nums_gene):
    
    # generate dataset with first nums_gene features
    df_sample = df[data.columns[0:nums_gene + 1]]

    # create both patient/gene id table to map patient/gene information
    df_pid = pd.DataFrame({
        'patients': df_sample['patients'],
        'id': np.arange(1,df_sample.shape[0] + 1)
    })
    
    gene = np.array(df_sample.columns[1:nums_gene + 1])
    df_gid = pd.DataFrame({
        'gene': gene,
        'id': np.arange(1,nums_gene + 1)
    })
    
    df_sample.columns = np.arange(len(df_sample.columns))
    df_sample.rename(columns={0:'pid'}, inplace=True)
    df_sample['pid'] = np.arange(1,df_sample.shape[0] + 1)
    
    # create long version of gene dataset
    df_long = pd.melt(df_sample,id_vars=['pid'], value_vars= df_sample.columns[1:nums_gene + 1])
    
    return df_sample, df_pid, df_gid, df_long

data_sample, pid, gid, data_sample_long = sample_data(data, 1000)
#data_sample_long


C:\Users\raoki\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
C:\Users\raoki\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [5]:
# generate SVD latent variables

from scipy.sparse.linalg import svds

#From https://github.com/beckernick/matrix_factorization_recommenders/blob/master/matrix_factorization_recommender.ipynb
def svd_latent_variable(df_long, nums_feature):
    R_df = df_long.pivot(index = 'pid', columns ='variable', values = 'value')
    R = R_df.as_matrix()
    U, sigma, Vt = svds(R, k = nums_feature)
    return U, R_df

latent, df = svd_latent_variable(data_sample_long, 20)


C:\Users\raoki\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [6]:
latent.shape

(100, 20)

In [7]:
# generate pmf(Probabilistic Matrix Factorization) latent variables

import matplotlib.pyplot as plt
import numpy as np
from LoadData import load_rating_data, spilt_rating_dat
from sklearn.model_selection import train_test_split
from ProbabilisticMatrixFactorization import PMF

pmf = PMF()
pmf.set_params({"num_feat": 100, "epsilon": 0.1, "_lambda": 0.1, "momentum": 0.8, "maxepoch": 5, "num_batches": 100,
                    "batch_size": 1000})
#pmf.set_params({"num_feat": 200})

ratings = np.array(data_sample_long, dtype = np.float)
print(len(np.unique(ratings[:, 0])), len(np.unique(ratings[:, 1])), pmf.num_feat)
train, test = train_test_split(ratings, test_size=0.2)  # spilt_rating_dat(ratings)
    
pmf.fit(train, test)
print(pmf.w_Item)
print(ratings.shape)
print(pmf.w_User.shape)
#print(pmf.w_User)
print(pmf.w_Item.shape)

100 1000 100
Training RMSE: 0.096148, Test RMSE 0.093297
Training RMSE: 0.090230, Test RMSE 0.087373
Training RMSE: 0.084749, Test RMSE 0.081841
Training RMSE: 0.079676, Test RMSE 0.076679
Training RMSE: 0.074978, Test RMSE 0.071859
[[-0.04761031  0.01180227 -0.05483487 ... -0.03580782 -0.11047747
  -0.04622762]
 [-0.05103023 -0.00419951 -0.03459323 ... -0.01815016  0.06387534
  -0.00187301]
 [-0.01931444  0.00219717  0.0235348  ...  0.00317385 -0.11518256
  -0.05249063]
 ...
 [-0.08363739  0.0952446   0.00535879 ...  0.05203544 -0.12857286
  -0.04593316]
 [-0.01454779 -0.1120015   0.13001167 ...  0.0668697   0.05918946
  -0.10925054]
 [-0.20855606 -0.01463089 -0.06954046 ...  0.08216036  0.13804424
   0.30224183]]
(100000, 3)
(101, 100)
(1001, 100)


In [8]:
print(pmf.w_User)
#print(pmf.w_User)
print(pmf.w_Item.shape)

[[-0.06940156 -0.16560991 -0.00646531 ... -0.05549272 -0.11312631
   0.19085001]
 [-0.04735175  0.02613067 -0.09240632 ... -0.11315014  0.00915784
  -0.04562747]
 [-0.01812339  0.03349816  0.06900455 ...  0.01010115  0.00115687
  -0.0774444 ]
 ...
 [-0.07101909 -0.00539553 -0.06421273 ... -0.12090779 -0.06018153
   0.02965691]
 [ 0.00862873  0.06857868  0.08390519 ... -0.00308781 -0.06230102
   0.07331187]
 [ 0.01339349  0.03358302  0.01391395 ...  0.01615677 -0.0046655
   0.06742813]]
(1001, 100)


In [9]:
format(pmf.w_User[1,1], '.15f')

'0.026130674041501'

In [10]:
data_sample_long['variable'] = data_sample_long['variable'] - 1


In [11]:
# generate latent variables using FactorAnalysis
from sklearn.decomposition import FactorAnalysis
X = np.array(data_sample_long.pivot(index = 'pid', columns ='variable', values = 'value'))
transformer = FactorAnalysis(n_components=200, random_state=0)
X_transformed = transformer.fit_transform(X)
X_transformed.shape

(100, 100)

In [12]:
# combine the class label with latent variables

data_label = pd.read_csv("data/tcga_cli.txt", sep = ";")
new_df = pd.DataFrame(latent)
new_df['label'] = np.array(data_label['new_tumor_event_dx_indicator'])
new_df

FileNotFoundError: File b'data/tcga_cli.txt' does not exist